<p><font size="6" color='grey'> <b>

Generative KI. Verstehen. Anwenden. Gestalten.
</b></font> </br></p>

<p><font size="5" color='grey'> <b>
Lokale und Open Source Modelle
</b></font> </br></p>


---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul
from genai_lib.utilities import (
    check_environment,
    get_ipinfo,
    setup_api_keys,
    mprint,
    install_packages,
    mermaid,
    get_model_profile,
    extract_thinking,
    load_prompt
)
print()
check_environment()
print()
get_ipinfo()

In [ ]:
#@title 🛠️ Installationen { display-mode: "form" }
install_packages(['pypdf'])

<p><font color='black' size="5">
⏸️ 5-Minuten-Check:
</font></p>

**Ziel:** Prüfen, ob du das vorherige Kapitel verstanden hast – nicht, ob es gerade läuft.

**Aufgabe** (5 Minuten, ohne Vorlage):

Rekonstruiere die zentrale Idee oder Code-Struktur des letzten Abschnitts selbstständig
(kein Copy & Paste, kein Nachschlagen).

Wähle eine der folgenden Optionen:

+ Erkläre in 1–2 Sätzen, was hier konzeptionell passiert.

+ Verändere eine Kleinigkeit (z. B. Prompt, Parameter, Reihenfolge) und beschreibe die Auswirkung.

+ Markiere eine Stelle, die du nicht sicher erklären kannst, und formuliere eine konkrete Frage dazu.

**Hinweis:**
Nicht alles muss „fertig“ oder „korrekt“ sein. Entscheidend ist, wo dein Verständnis gerade endet

# 1 | Einführung
---


Lokale und Open Source Modelle bieten eine Alternative zu Cloud-basierten KI-Diensten wie OpenAI. Im Gegensatz zu API-basierten Diensten werden diese Modelle direkt auf dem lokalen System ausgeführt, was Vorteile in Bezug auf Datenschutz, Kosten und Anpassungsmöglichkeiten bietet.



<p><font color='black' size="5">
Vor- und Nachteile lokaler Modelle
</font></p>

- **Vorteile**: Datenschutz, keine API-Kosten, keine Internetverbindung erforderlich, volle Kontrolle
- **Nachteile**: Höhere Hardwareanforderungen, begrenzte Modellleistung auf Standard-Hardware, Installations- und Wartungsaufwand



<p><font color='black' size="5">
Anwendungsbereiche lokaler Modelle
</font></p>

- Entwicklung und Prototyping
- Datenschutzkritische Anwendungen
- Offline-Anwendungen
- Kostensensitive Projekte



# 2 | Ollama
---

<img src="https://ollama.com/public/ollama.png" class="logo" width="120"/>





Ollama ist ein Tool, das die einfache Installation und Nutzung von Open Source LLMs auf dem eigenen Computer ermöglicht. Es bietet eine benutzerfreundliche Schnittstelle zum Herunterladen, Verwalten und Ausführen verschiedener Sprachmodelle.

[Ollama](https://ollama.com/)

In [ ]:
#@title 🧜‍♀️ Prozess-Diagramm { display-mode: "form" }

diagram = """
graph TD
    subgraph Colab ["<b>Google Colab Runtime</b>"]
        direction TB

        Code["Python Code<br/><small>Notebook-Zelle</small>"]

        subgraph Backend ["Hintergrund"]
            Ollama["Ollama Server<br/><small>localhost:11434</small>"]
            Models["Lokale Modelle"]
        end

        GPU["GPU<br/><small>T4 / A100</small>"]
    end

    subgraph Extern ["Extern"]
        Registry["Ollama Registry"]
    end

    %% Haupt-Workflow (nummeriert)
    Code -->|"#1 API Call"| Ollama
    Ollama -->|"#2 laden"| Models
    Ollama -->|"#3 Inferenz"| GPU
    GPU -->|"#4 Ergebnis"| Ollama
    Ollama -->|"#5 Response"| Code

    %% Setup (gestrichelt)
    Registry -.->|"ollama pull"| Models

    %% Styling
    style Colab fill:#f8f9fa,stroke:#495057,stroke-width:2px
    style GPU fill:#d4edda,stroke:#28a745,stroke-width:2px
    style Ollama fill:#e3f2fd,stroke:#1976d2,stroke-width:2px
    style Registry fill:#fff3cd,stroke:#ffc107
"""
mermaid(diagram)


<p><font color='black' size="5">
Installation von Ollama
</font></p>

In [ ]:
# Installation in Linux/Colab
!sudo apt update
!sudo apt install -y pciutils
!sudo apt-get install zstd
!curl -fsSL https://ollama.com/install.sh | sh

<p><font color='black' size="5">
Starten des Ollama-Servers
</font></p>

In [ ]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(10)  # Kurz warten, bis der Server hochgefahren ist

<p><font color='black' size="5">
Modelle herunterladen und verwalten
</font></p>

In [ ]:
# Modell herunterladen
!ollama pull llama3:latest

In [ ]:
# Modell herunterladen
!ollama pull qwen3:latest

In [ ]:
# Verfügbare Modelle anzeigen
!ollama list

# 3 | Lokale Modelle und LangChain
---

LangChain bietet eine nahtlose Integration mit Ollama, wodurch lokale Modelle in komplexen Anwendungen eingesetzt werden können.



<p><font color='black' size="5">
Einfache Verwendung von Ollama mit LangChain
</font></p>

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain.chat_models import init_chat_model

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher und humorvoller KI-Assistent. Antworte sachlich und kurz."),
    ("human", "Frage: {user_input}\n\nAntwort: Denke Schritt für Schritt.")
])

# Model
llm = init_chat_model("ollama:llama3", temperature=0.1)

# Parser
parser = StrOutputParser()

# Chain zusammensetzen und ausführen
chain = prompt | llm | parser

<p><font color='black' size="5">
Beispiele Llama 3
</font></p>

In [ ]:
# Beispiel 1: Gesundheitsfrage
response = chain.invoke({"user_input": "Was ist der einfachste Weg Gewicht abzunehmen?"})
mprint("## 🤖 Local LLM")
mprint(response)

<p><font color='black' size="5">
Textgenerierung (aus M05)
</font></p>

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
# Abschnitt 3: Chain-Komponenten initialisieren

system_prompt = """
Du bist ein hilfreicher KI-Assistent.
"""

# Die Chat-Prompt definieren mit drei Variablen: system_prompt, history, user_input
prompt = ChatPromptTemplate.from_messages([
        ("system", "{system_prompt}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{user_input}")
])

# OutputParser definieren
parser = StrOutputParser()

# Chain erstellen
chain = prompt | llm | parser

In [ ]:
# Abschnitt 4: Funktionen definieren

def chat(system_prompt, chat_history, user_input):
    """Führt eine einzelne Interaktion mit der KI durch."""

    # Aufruf der Kette
    parameter = {
        'system_prompt': system_prompt,
        'chat_history': chat_history,
        'user_input': user_input
    }
    response = chain.invoke(parameter)

    # Ausgabe der Ergebnisse
    mprint("### 🧑‍🦱 Mensch:")
    mprint(user_input)

    mprint("### 🤖 KI:")
    mprint(response)

    # Memory-Management - Erweiterung der Historie
    chat_history.extend([
        HumanMessage(content=user_input),
        AIMessage(content=response)
    ])

    return chat_history

In [ ]:
# Abschnitt 5: Hauptprogramm

# Historie wird initialisiert
chat_history = [SystemMessage(content=system_prompt)]

# Liste mit user_input
user_input_list = [
    "Mein Name ist Ralf",
    "Warum ist der Himmel blau?",
    "Und warum ist er manchmal rot?",
    "Wie ist mein Name?"
]

# Chain-Aufruf in einer Schleife
for user_input in user_input_list:
    chat_history = chat(system_prompt, chat_history, user_input)

In [ ]:
chat_history


<p><font color='black' size="5">
Textklassifizierung (aus M05)
</font></p>

In [ ]:
# Konstanten definieren
system_prompt = """
Du bist ein präziser Namensextraktor.

**Aufgabe:** Extrahiere ausschließlich die Namen aus dem gegebenen Text.

**Was zu extrahieren ist:**
- Personennamen (Vor- und Nachnamen)
- Unternehmensnamen
- Produktnamen

**Ausgabeformat:**
- Ein Name pro Zeile
- Keine Nummerierung
- Keine Aufzählungszeichen
- Keine zusätzlichen Wörter oder Erklärungen
- Keine Duplikate

**Wichtig:**
- Gib NUR die extrahierten Namen aus
- Formuliere den Originaltext NICHT um
- Füge KEINE weiteren Informationen hinzu

**Beispiel:**
Input: "Peter Müller arbeitet bei der Siemens AG und entwickelt das NextPhone."
Output:
Peter Müller
Siemens AG
NextPhone
"""

user_input = """
Anna und Sarah sind seit dem Studium befreundet. Nach ihrem Abschluss hatten sie ein Vorstellungsgespräch bei Frau Weber für eine Stelle bei der Technovision GmbH, deren Hauptprodukt Futurtech heißt und von der Programmiererin Maria Schmidt entwickelt wurde. Nach dem erfolgreichen Gespräch trafen sie sich mit ihren Freunden Rafael und Aida zum Feiern. An ihrem ersten Arbeitstag lernten sie drei weitere Mitarbeiter kennen: Richard, Lisa und Pia. Später kam noch Matthias Müller zum Team dazu.
"""

In [ ]:
# Model
llm = init_chat_model("ollama:llama3", temperature=0.1)

# Parser
parser = StrOutputParser()

# Chain zusammensetzen und ausführen
chain = prompt | llm | parser

In [ ]:
# Hauptprogramm
chat_history = [SystemMessage(content=system_prompt)]

# Namen extrahieren und anzeigen
parameter = {}
parameter['system_prompt'] = system_prompt
parameter['chat_history'] = chat_history
parameter['user_input'] = user_input
response = chain.invoke(parameter)

# Ausgabe
mprint("## 📛 Gefundene Namen:")
mprint("---")
print(response)

<p><font color='black' size="5">
Textzusammenfassung (aus M05)
</font></p>

In [ ]:
# Importe für Dokument-Verarbeitung
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document

In [ ]:
# Chat-Komponenten initialisieren

# Chat-Prompt-Template erstellen
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent, der präzise Zusammenfassungen erstellt."),
    ("human", """Schreibe eine kurze Zusammenfassung der präsentierten Informationen.
    Schreibe die Zusammenfassung auf Deutsch.

{user_input}

ZUSAMMENFASSUNG:""")
])

# Modell definieren (Kurznotation: "provider:model")
llm = init_chat_model("ollama:qwen3", temperature=0.3)

# Parser definieren
parser = StrOutputParser()

# Kette erstellen
chain = prompt | llm | parser

In [ ]:
# Funktionen definieren
def load_document(url: str):
    """Lädt ein PDF-Dokument von einer URL und splittet es in Chunks"""
    loader = PyPDFLoader(url)
    return loader.load_and_split()

In [ ]:
# Hauptprogramm
url = "https://arxiv.org/pdf/1706.03762"

# Dokument laden
docs = load_document(url)

# Alle Chunks zu einem Text zusammenfügen
text = " ".join([doc.page_content for doc in docs])

# Zusammenfassung generieren
response = chain.invoke({"user_input": text})

# Ausgabe
mprint("## ✨ Zusammenfassung:")
mprint("---")
mprint(response)


# 4 | Open Source vs. Closed Source
---

<p><font color='black' size="5">
Open Source Modelle
</font></p>



Open Source KI-Modelle zeichnen sich durch ihre frei zugänglichen Gewichte und Architekturen aus. Diese Modelle werden unter Lizenzen wie MIT, Apache oder Creative Commons veröffentlicht, die es Nutzern erlauben, den Code einzusehen, zu modifizieren und für eigene Zwecke anzupassen.

**Vorteile von Open Source Modellen:**

- **Transparenz**: Einsicht in die Funktionsweise und Trainingsmethoden
- **Anpassbarkeit**: Möglichkeit zum Fine-Tuning für spezifische Anwendungsfälle
- **Keine API-Kosten**: Nach dem Download entstehen keine laufenden Nutzungskosten
- **Datenschutz**: Vollständige Kontrolle über die Datenverarbeitung
- **Community-Unterstützung**: Regelmäßige Verbesserungen durch die Community

**Herausforderungen:**

- **Hardwareanforderungen**: Größere Modelle benötigen leistungsstarke Hardware
- **Expertise**: Technisches Know-how für Installation und Optimierung nötig
- **Qualitätsunterschiede**: Leistung kann bei gleicher Größe hinter kommerziellen Modellen zurückbleiben



<p><font color='black' size="5">
Closed Source Modelle
</font></p>

Closed Source Modelle wie GPT-4 von OpenAI oder Claude von Anthropic werden nur über APIs angeboten, ohne Zugriff auf die zugrunde liegenden Modellgewichte.

**Vorteile von Closed Source Modellen:**

- **Leistungsstärke**: Oft fortschrittlicher als vergleichbare Open Source Alternativen
- **Einfache Nutzung**: Keine lokale Installation oder Wartung nötig
- **Regelmäßige Updates**: Automatische Verbesserungen ohne eigenen Aufwand
- **Skalierbarkeit**: Flexibel an unterschiedlichen Bedarf anpassbar

**Nachteile:**

- **Kosten**: Nutzungsbasierte Gebühren
- **Abhängigkeit**: Angewiesenheit auf externe Dienste und deren Verfügbarkeit
- **Datenschutzbedenken**: Daten verlassen das eigene System
- **Eingeschränkte Anpassbarkeit**: Beschränkte Möglichkeiten zur Modifikation



# 5 | Lokal vs. Cloud
---


Beim Arbeiten mit generativer KI gibt es grundsätzlich zwei Möglichkeiten: Modelle können **lokal auf dem eigenen Rechner** ausgeführt werden oder es werden **Cloud-Dienste** wie ChatGPT, Claude oder Google Gemini genutzt.

**Cloud-Ausführung**

Bei Cloud-Diensten läuft das KI-Modell auf Servern des Anbieters. Die Anfrage wird über das Internet gesendet, und der Server schickt die Antwort zurück.

**Vorteile:**
- Sofort nutzbar, keine Installation nötig
- Zugang zu den leistungsstärksten Modellen (GPT-4, Claude Opus, Gemini Ultra)
- Keine teure Hardware erforderlich
- Modelle werden automatisch aktualisiert

**Nachteile:**
- Daten verlassen den eigenen Rechner
- Laufende Kosten (oft pro Anfrage oder Abo)
- Internetverbindung erforderlich
- Abhängigkeit vom Anbieter

**Lokale Ausführung**

Bei lokaler Ausführung läuft das Modell direkt auf dem eigenen Computer – zum Beispiel mit Tools wie Ollama, LM Studio oder llama.cpp.

**Vorteile:**
- Volle Datenkontrolle – nichts verlässt den Rechner
- Keine laufenden Kosten nach dem Download
- Funktioniert offline
- Ideal für sensible oder vertrauliche Daten

**Nachteile:**
- Leistungsstarke Hardware nötig (besonders GPU mit viel VRAM)
- Lokale Modelle sind meist kleiner und weniger leistungsfähig
- Einrichtung erfordert technisches Know-how
- Updates müssen selbst durchgeführt werden

# 6 | Lizenzierung und rechtliche Aspekte
---

Die Lizenzen von Open Source Modellen unterscheiden sich erheblich und beeinflussen deren Einsatzmöglichkeiten:

**MIT Lizenz (z. B. OpenLLaMA, GPT-NeoX)**
- Sehr liberal, erlaubt kommerzielle Nutzung ohne Einschränkungen  
- Erfordert lediglich die Nennung des ursprünglichen Urhebers  
- Keine Weitergabelizenzverpflichtung (nicht copyleft)  
- Einfach und weit verbreitet in Open-Source-Projekten  

**Apache 2.0 (z. B. Gemma, Falcon)**
- Sehr liberal, erlaubt kommerzielle Nutzung ohne starke Einschränkungen  
- Erfordert Hinweis auf ursprüngliches Urheberrecht  
- Keine Weitergabelizenzverpflichtung (nicht copyleft)  
- Enthält eine Klausel zum Schutz vor Patentstreitigkeiten  

**Meta Llama Lizenz**
- Erlaubt kommerzielle Nutzung mit Einschränkungen (z. B. Nutzerschwellen)  
- Enthält spezifische Bedingungen für verantwortungsvolle Nutzung  
- Verbietet bestimmte Anwendungsfälle wie Desinformation oder illegale Aktivitäten  
- Nicht so frei wie Apache oder MIT, aber offener als proprietäre Modelle  

**Mistral AI Lizenz**
- Erlaubt kommerzielle und nicht-kommerzielle Nutzung  
- Enthält ethische Nutzungsbeschränkungen  
- Unterschiedliche Bedingungen für verschiedene Modellversionen  
- Im Allgemeinen offener als die Meta Llama Lizenz  

**Creative Commons (z. B. BLOOM unter CC-BY-SA)**
- Verschiedene Varianten mit unterschiedlichem Freiheitsgrad  
- **CC-BY**: Erlaubt kommerzielle Nutzung, solange der Urheber genannt wird  
- **CC-BY-SA**: Weitergabe nur unter gleichen Lizenzbedingungen erlaubt (copyleft)  
- **CC-BY-NC**: Keine kommerzielle Nutzung erlaubt  
- **CC-BY-ND**: Keine Bearbeitung oder Ableitung erlaubt  
- Häufig für Open-Access-Modelle genutzt, aber mit Einschränkungen je nach Variante  

<br>

**Disclaimer**    
Die oben aufgeführten Informationen zu Lizenzmodellen dienen ausschließlich allgemeinen Informationszwecken und stellen keine rechtliche Beratung dar. Die Bedingungen einzelner Lizenzen können sich ändern, und ihre Auslegung kann von der jeweiligen Rechtsprechung abhängen.

Vor der Nutzung oder Weitergabe eines KI-Modells sollten die offiziellen Lizenzdokumente sorgfältig geprüft und bei Unsicherheiten eine juristische Beratung eingeholt werden. Wir übernehmen keine Haftung für die Vollständigkeit, Richtigkeit oder Anwendbarkeit der bereitgestellten Informationen.



# A | Aufgabe


Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.

<p><font color='black' size="5">
Vergleich von Open Source Modellen
</font></p>

Installieren Sie Ollama auf Ihrem lokalen System und vergleichen Sie die Leistung von mindestens zwei verschiedenen Modellen (z.B. Llama2 und Gemma) bei der Beantwortung derselben Prompts.

Vergleichen Sie die Antworten der Modelle. Kriterien: Umfang, Qualität, Antwortzeit, ...


<p><font color='black' size="5">
Chat-Anwendung mit lokalem LLM
</font></p>


Entwickeln Sie eine einfache Gradio-basierte Chat-Anwendung, die ein lokales Modell über Ollama nutzt. Die Anwendung soll:

```python
import gradio as gr
from langchain_ollama import OllamaLLM
from langchain_core.messages import HumanMessage, AIMessage
import os

# Ollama-LLM initialisieren
llm = OllamaLLM(model="gemma", temperature=0.7)

# Chat-Historie
chat_verlauf = []

def chat_antwort(nachricht, verlauf):
    """
    Verarbeitet die Nutzereingabe und generiert eine Antwort vom LLM
    """
    # Aktuelle Nachricht zur Historie hinzufügen
    if nachricht:
        verlauf.append((nachricht, ""))
        
        # Chat-Kontext erstellen (begrenzt auf die letzten 5 Nachrichten für Effizienz)
        kontext = ""
        for i in range(max(0, len(verlauf)-5), len(verlauf)):
            if i < len(verlauf)-1:  # Alle vorherigen Nachrichten
                kontext += f"Mensch: {verlauf[i][0]}\nAssistent: {verlauf[i][1]}\n"
            else:  # Aktuelle Nachricht
                kontext += f"Mensch: {verlauf[i][0]}\nAssistent: "
        
        # Antwort vom Modell generieren
        antwort = llm.invoke(kontext)
        
        # Antwort zur Historie hinzufügen
        verlauf[-1] = (nachricht, antwort)
    
    return "", verlauf

# Gradio-Interface erstellen
with gr.Blocks(title="Lokaler KI-Assistent") as demo:
    gr.Markdown("# Chat mit einem lokalen KI-Modell")
    gr.Markdown("Dieser Chat verwendet das Gemma-Modell über Ollama")
    
    chatbot = gr.Chatbot(label="Chat-Verlauf")
    msg = gr.Textbox(label="Ihre Nachricht", placeholder="Schreiben Sie hier Ihre Nachricht...")
    
    msg.submit(chat_antwort, [msg, chatbot], [msg, chatbot])
    
    gr.Markdown("### Hinweise zur Nutzung:")
    gr.Markdown("- Die Antworten werden lokal auf Ihrem System generiert")
    gr.Markdown("- Keine Daten werden an externe Server gesendet")
    gr.Markdown("- Die Antwortgeschwindigkeit hängt von Ihrer Hardware ab")

# Anwendung starten
demo.launch()
```

Testen & erweitern Sie die Anwendung:
1. Testen Sie den Code, korrigieren Sie etwaige Fehler.
2. Modellauswahl (zwischen verschiedenen installierten Ollama-Modellen)
3. Parameter-Einstellungen (Temperatur, Top-K, Top-P)
4. Speichern und Laden von Chat-Verläufen
5. Möglichkeit, die generierten Antworten zu bewerten (Daumen hoch/runter)

<p><font color='black' size="5">
RAG-System mit lokalem LLM
</font></p>



Entwickeln Sie ein einfaches Retrieval-Augmented Generation (RAG) System, das lokale PDF-Dokumente einliest und die Beantwortung von Fragen mit einem lokalen Modell ermöglicht:

```python
from langchain_ollama import OllamaLLM
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
import gradio as gr
import os

# Modell initialisieren
llm = OllamaLLM(model="llama2", temperature=0.1)

# Embedding-Modell laden (lokal, ohne API-Calls)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Vektorstore für Dokumente
vector_store = None

def dokumente_laden(dateien):
    """Lädt PDF-Dokumente und erstellt eine Vektordatenbank"""
    global vector_store
    
    dokumente = []
    for datei in dateien:
        loader = PyPDFLoader(datei.name)
        dokumente.extend(loader.load())
    
    # Dokumente in kleinere Chunks aufteilen
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    chunks = text_splitter.split_documents(dokumente)
    
    # Vektordatenbank erstellen
    vector_store = FAISS.from_documents(chunks, embeddings)
    
    return f"{len(chunks)} Textabschnitte aus {len(dateien)} Dokumenten geladen"

def frage_beantworten(frage):
    """Beantwortet eine Frage basierend auf den geladenen Dokumenten"""
    if vector_store is None:
        return "Bitte laden Sie zuerst Dokumente hoch."
    
    # Relevante Dokumente abrufen
    docs = vector_store.similarity_search(frage, k=3)
    
    # Kontext erstellen
    kontext = "\n\n".join([doc.page_content for doc in docs])
    
    # Prompt erstellen
    prompt_vorlage = """
    Du bist ein hilfreicher Assistent, der Fragen basierend auf gegebenen Informationen beantwortet.
    
    Kontext:
    {kontext}
    
    Frage: {frage}
    
    Deine Antwort sollte sich auf die im Kontext gegebenen Informationen beschränken.
    Wenn du keine Antwort im Kontext findest, sage ehrlich, dass du es nicht weißt.
    """
    
    prompt = PromptTemplate(
        template=prompt_vorlage,
        input_variables=["kontext", "frage"]
    )
    
    # Antwort vom LLM bekommen
    antwort = llm.invoke(prompt.format(kontext=kontext, frage=frage))
    
    return antwort

# Gradio-Interface
with gr.Blocks() as demo:
    gr.Markdown("# Lokales Frage-Antwort-System mit RAG")
    
    with gr.Tab("Dokumente laden"):
        file_input = gr.File(file_count="multiple", label="PDF-Dokumente")
        upload_button = gr.Button("Dokumente laden")
        upload_output = gr.Textbox(label="Status")
        upload_button.click(dokumente_laden, inputs=[file_input], outputs=[upload_output])
    
    with gr.Tab("Fragen stellen"):
        question_input = gr.Textbox(label="Ihre Frage zum Dokument")
        answer_button = gr.Button("Frage stellen")
        answer_output = gr.Textbox(label="Antwort")
        answer_button.click(frage_beantworten, inputs=[question_input], outputs=[answer_output])

demo.launch()
```

Testen und erweitern Sie das System:
1. Testen & ggf. korrigieren des Code
2. Unterstützung für weitere Dateiformate (DOCX, TXT, HTML)
3. Möglichkeit, die Qualität der Antworten zu verbessern durch:
   - Modellauswahl
   - Parameteroptimierung
   - Verbesserung des Prompts
4. Implementierung einer Möglichkeit zur Quellennachverfolgung (aus welchem Teil des Dokuments stammt die Information)
5. Visualisierung der relevanten Textpassagen im Dokument